In [1]:
!pip install langchain langchain-huggingface huggingface_hub
!pip install sentence-transformers chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 11.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: nvidia-nvji

In [2]:
# pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"

device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

2025-05-18 19:17:36.551763: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747595856.744363      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747595856.796828      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

## Setup

Let's start by importing the necessary libraries and setting up our environment.

## Basic Chain of Thought Prompting

Let's start with a simple example to demonstrate the difference between a standard prompt and a Chain of Thought prompt.

In [3]:
%pip install transformers langchain langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22
Note: you may need to restart the kernel to use updated packages.


In [4]:
from datasets import load_dataset

ds = load_dataset("qiaojin/PubMedQA", "pqa_artificial")



README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

In [5]:
ds = ds['train'].select_columns(["question", "context", "long_answer", "final_decision"])
ds = ds.select(range(30))
print(ds)

small_dataset = ds.to_list()

print(f"Loaded {len(small_dataset)} processed samples.")

Dataset({
    features: ['question', 'context', 'long_answer', 'final_decision'],
    num_rows: 30
})
Loaded 30 processed samples.


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = ["\n\n".join(doc["context"]['contexts']) for doc in small_dataset]

embeddings = embedding_model.embed_documents(texts)

vector_store = FAISS.from_texts(texts, embedding_model)

print("Embeddings stored in FAISS.")

/tmp/ipykernel_35/420776512.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings stored in FAISS.


In [26]:
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain.prompts import PromptTemplate
import re

def retrieve_context(state):
    print("retriveing")
    """Retrieve relevant context using FAISS."""
    retrieved_docs = vector_store.similarity_search(state["question"], k=3)
    state["context"] = "\n".join([doc.page_content for doc in retrieved_docs])
    
    return state

retrieval = RunnableLambda(retrieve_context)



def generate_response(prompt):
    print("generating")
    messages = [{"role": "user", "content": f"{prompt}"}]
    input_text = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=350, temperature=0.6, top_p=0.92, do_sample=True)
    
    return  tokenizer.decode(outputs[0], skip_special_tokens=True).replace("'", "")

generation = RunnableLambda(generate_response)


In [27]:
standard_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=""""Context: {context} 
Question: {question}"""
)

# Chain of Thought Prompt
cot_prompt = PromptTemplate(
input_variables=["question","context"],
template="""

Context: 
-------
{context}

Guidelines
----------
You are a medical AI assistant.
Provide accurate step by step  detailed answer concisely using provided context.
.
Question:
{question}
    """
)

In [20]:
# nstruction – "Generate a detailed summary of the following article."
# Persona/Role (optional) – "You are an expert science communicator."
# Tone (optional) – "Use an informative and engaging tone."
# Context (optional) – "The article discusses recent advancements in renewable
# energy technologies, focusing on solar and wind power."
# Input Data/Question – "Here is the article text: [insert article text]."
# Constraints (optional) – "Keep the summary under 200 words and avoid
# technical jargon."
# Output Indicator (optional) – "Present the summary in bullet points."

# # Chain of Thought Prompt
# cot_prompt = PromptTemplate(
# input_variables=["question","context"],
# template="""
# Generate a detailed answer for the following question
# You are a medical AI assistant.
# Use an informative and engaging tone
# Context : {context}
# Here is the question: {question}
# Keep the summary under 200 words and avoid technical jargon.
# Present the summary in bullet points."""
# )


In [29]:
cot_prompt_based_pipeline = retrieval | cot_prompt | generation

zero_shot_prompt_based_pipeline = retrieval | standard_prompt | generation


# Example question
question_data = {"question": "Are group 2 innate lymphoid cells ( ILC2s ) increased in chronic rhinosinusitis with nasal polyps or eosinophilia?"}

cot_response = cot_prompt_based_pipeline.invoke(question_data)

# # Execute pipeline
zero_shot_response = zero_shot_prompt_based_pipeline.invoke(question_data)

print("\n🔹 Final RAG Response:", zero_shot_response.replace("\\n", "\n"))
print("\n🔹 Final RAG Response:", cot_response.replace("\\n", "\n"))

retriveing
generating
retriveing
generating

🔹 Final RAG Response: user
text="Context: Chronic rhinosinusitis (CRS) is a heterogeneous disease with an uncertain pathogenesis. Group 2 innate lymphoid cells (ILC2s) represent a recently discovered cell population which has been implicated in driving Th2 inflammation in CRS; however, their relationship with clinical disease characteristics has yet to be investigated.

The aim of this study was to identify ILC2s in sinus mucosa in patients with CRS and controls and compare ILC2s across characteristics of disease.

A cross-sectional study of patients with CRS undergoing endoscopic sinus surgery was conducted. Sinus mucosal biopsies were obtained during surgery and control tissue from patients undergoing pituitary tumour resection through transphenoidal approach. ILC2s were identified as CD45(+) Lin(-) CD127(+) CD4(-) CD8(-) CRTH2(CD294)(+) CD161(+) cells in single cell suspensions through flow cytometry. ILC2 frequencies, measured as a perce

In [13]:
!pip install evaluate rouge_score bleu meteor evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached bleu-0.3.tar.gz (5.2 kB)
  Preparing metadata (setup.py) ... done
  Using cached meteor-2.0.18-py3-none-any.whl.metadata (8.5 kB)
  Using cached efficiency-2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached bgzip-0.5.0.tar.gz (100 kB)
  Preparing metadata (setup.py) ... done
  Using cached biom-format-2.1.16.tar.gz (11.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.3 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 MB 20.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 42.0 MB/s

In [30]:
import re
import string
from collections import Counter
from tqdm import tqdm
from dotenv import load_dotenv
import os
import pandas as pd
from collections import Counter
import re
import string
import unicodedata
import numpy as np 
import evaluate
from rouge_score import rouge_scorer

# Normalize text for EM/F1
def normalize_text(text):
    def remove_articles(s):
        return re.sub(r'\b(a|an|the)\b', ' ', s)

    def white_space_fix(s):
        return ' '.join(s.split())

    def remove_punc(s):
        return ''.join(ch for ch in s if ch not in string.punctuation)

    def lower(s):
        return s.lower()

    def unicode_to_ascii(s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    return white_space_fix(remove_articles(remove_punc(lower(unicode_to_ascii(text)))))
    
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

def get_metrics(df):

     

    # BLEU
    bleu_result = bleu.compute(predictions=predictions, references=[[a] for a in answers])
    # ROUGE
    rouge_result = rouge.compute(predictions=predictions, references=answers)

    return em_scores, f1_scores, bleu_result, rouge_result

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    gt_tokens = normalize_answer(ground_truth).split()
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    return 2 * (precision * recall) / (precision + recall)

def evaluate(dataset, prompting_type):
    f1_scores = []
    for item in small_dataset:
        print("1..")
        question = {"question":item["question"]}
        prediction = cot_prompt_based_pipeline.invoke(question).replace("\\n", "")
        ground_truth = item['long_answer']
        
        pred_tokens = normalize_text(prediction).split()
        gt_tokens = normalize_text(ground_truth).split()
        
        common = Counter(pred_tokens) & Counter(gt_tokens)
        num_same = sum(common.values())
        if num_same == 0:
            return 0.0
        precision = num_same / len(pred_tokens)
        recall = num_same / len(gt_tokens)
        f1_scores.append(2 * precision * recall / (precision + recall))
    return  np.mean(f1_scores)
    
import random
import numpy as np

def eval_rouge(dataset, prompting_type=None):  # prompting_type is unused
    # Sample 500 items
    small_dataset = random.sample(dataset, 30)

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Score lists for each pipeline
    cot_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    zero_shot_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for item in small_dataset:
        question = {"question": item["question"]}
        ground_truth = item["long_answer"]

        # Get predictions
        cot_pred = cot_prompt_based_pipeline.invoke(question).replace("\\n", "")
        zs_pred = zero_shot_prompt_based_pipeline.invoke(question).replace("\\n", "")

        # CoT scores
        cot_result = scorer.score(ground_truth, cot_pred)
        cot_scores['rouge1'].append(cot_result['rouge1'].fmeasure)
        cot_scores['rouge2'].append(cot_result['rouge2'].fmeasure)
        cot_scores['rougeL'].append(cot_result['rougeL'].fmeasure)

        # Zero-shot scores
        zs_result = scorer.score(ground_truth, zs_pred)
        zero_shot_scores['rouge1'].append(zs_result['rouge1'].fmeasure)
        zero_shot_scores['rouge2'].append(zs_result['rouge2'].fmeasure)
        zero_shot_scores['rougeL'].append(zs_result['rougeL'].fmeasure)

    # Average results
    print("\nAverage ROUGE Scores over 500 samples:\n")

    print("**CoT Prompting**")
    print("Avg ROUGE-1:", np.mean(cot_scores['rouge1']))
    print("Avg ROUGE-2:", np.mean(cot_scores['rouge2']))
    print("Avg ROUGE-L:", np.mean(cot_scores['rougeL']))

    print("\n**Zero-Shot Prompting**")
    print("Avg ROUGE-1:", np.mean(zero_shot_scores['rouge1']))
    print("Avg ROUGE-2:", np.mean(zero_shot_scores['rouge2']))
    print("Avg ROUGE-L:", np.mean(zero_shot_scores['rougeL']))

# F1 Score
# def f1_score(prediction, ground_truth):
#     pred_tokens = normalize_text(prediction).split()
#     gt_tokens = normalize_text(ground_truth).split()
#     common = Counter(pred_tokens) & Counter(gt_tokens)
#     num_same = sum(common.values())
#     if num_same == 0:
#         return 0.0
#     precision = num_same / len(pred_tokens)
#     recall = num_same / len(gt_tokens)
#     return 2 * precision * recall / (precision + recall)

    
print(eval_rouge(small_dataset, "cot"))

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing
generating
retriveing